In [1]:
import heapq
import itertools
import time

class AdvancedOrderBook:
    def __init__(self):
        self.bids = []  
        self.asks = []  
        self.trades = []
        self.order_id_counter = itertools.count() # Monotonic counter for time priority

    def submit_order(self, side, qty, price=None, order_type='limit'):
        """
        Entry point for all orders. 
        1. Tries to match immediately (crossing the spread).
        2. If limit order has qty left, adds to book.
        """
        if order_type == 'market':
            # Market Buy: Willing to pay Infinity
            # Market Sell: Willing to sell for 0 (or -Infinity conceptually)
            limit_price = float('inf') if side == 'buy' else 0
        else:
            limit_price = price

        remaining_qty = self.match(side, qty, limit_price)

        if remaining_qty > 0 and order_type == 'limit':
            entry_id = next(self.order_id_counter)
            if side == 'buy':
                heapq.heappush(self.bids, [-limit_price, entry_id, remaining_qty])
            else:
                heapq.heappush(self.asks, [limit_price, entry_id, remaining_qty])
            
        return remaining_qty

    def match(self, side, qty, limit_price):
        """
        Walks the book to fill the incoming order.
        Returns: remaining_qty (if any)
        """
        remaining_qty = qty
        
        while remaining_qty > 0:
            if side == 'buy':
                if not self.asks: break
                best_price = self.asks[0][0]

                if limit_price < best_price: break
                
                best_order = self.asks[0]
            else:
                if not self.bids: break
                best_price = -self.bids[0][0] 
                
                if limit_price > best_price: break
                
                best_order = self.bids[0]

            # --- EXECUTION LOGIC ---
            trade_qty = min(remaining_qty, best_order[2])
            
            # Execution Price is always the RESTING order's price
            # (If I buy at market, I pay the specific Ask price at this level)
            exec_price = best_order[0] if side == 'buy' else -best_order[0]

            # Log Trade
            self.trades.append({
                'price': exec_price,
                'qty': trade_qty,
                'timestamp': time.time(),
                'side': side,
                'aggressor': 'market' if limit_price == float('inf') else 'limit'
            })

            remaining_qty -= trade_qty
            best_order[2] -= trade_qty

            # Clean up the Book (Remove filled orders)
            if best_order[2] == 0:
                if side == 'buy':
                    heapq.heappop(self.asks)
                else:
                    heapq.heappop(self.bids)
                    
        return remaining_qty

Validation test

In [2]:
def run_validation_test():
    ob = AdvancedOrderBook()

    print("--- 1. Submitting ASK Ladder ---")
    ob.submit_order('sell', 10, 101)
    ob.submit_order('sell', 20, 102)
    ob.submit_order('sell', 30, 103)

    print(f"Current Asks (Top): {ob.asks}")
    print("\n--- 2. Submitting MARKET BUY (Qty=60) ---")
    
    # Market Buy should walk up from 101 -> 102 -> 103
    ob.submit_order('buy', 60, order_type='market')

    print("\n--- 3. Validation Results ---")
    
    # Check 1: Is the Ask book empty?
    is_book_empty = len(ob.asks) == 0
    print(f"PASS: Ask Book is Empty? -> {is_book_empty}")

    # Check 2: Are the trades correct?
    print("Trade Log:")
    for t in ob.trades:
        print(f"  Price: {t['price']}, Qty: {t['qty']}")

    # Check 3: Logic Verification
    expected_trades = [
        (101, 10),
        (102, 20),
        (103, 30)
    ]
    
    match_success = True
    if len(ob.trades) != 3:
        match_success = False
    else:
        for i, (exp_p, exp_q) in enumerate(expected_trades):
            if ob.trades[i]['price'] != exp_p or ob.trades[i]['qty'] != exp_q:
                match_success = False
    
    print(f"\nOVERALL VALIDATION: {'SUCCESS' if match_success and is_book_empty else 'FAIL'}")

run_validation_test()

--- 1. Submitting ASK Ladder ---
Current Asks (Top): [[101, 0, 10], [102, 1, 20], [103, 2, 30]]

--- 2. Submitting MARKET BUY (Qty=60) ---

--- 3. Validation Results ---
PASS: Ask Book is Empty? -> True
Trade Log:
  Price: 101, Qty: 10
  Price: 102, Qty: 20
  Price: 103, Qty: 30

OVERALL VALIDATION: SUCCESS
